In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import sisepuede_file_structure as sfs
import support_classes as sc
import support_functions as sf
import importlib
import sql_utilities as squ
import time
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2135: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{CH}_4 Stationary Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2135: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{CO}_2 Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2135: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{N}_2\text{O} Stationary Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2135

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


# There's no good crosswalk for the regions used in the IPCC NGHGI guidance
- there is one reference in 19R V5C6 Table 6A.6
- the AR5 WG3 Annex II (https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-ii.pdf) has a list in A.II.2.2
- I pull this list in here and try to build a manual merge to regions, collapsing some from RG10 using assumptions shown in dictionaries below

In [74]:
# read in text file
path_text = "/Users/jsyme/Documents/IPCC Documents/AR5/ipcc_wg3_ar5_annex-ii_A.II.2.2.txt"
with open(path_text, "r") as file:
    text = file.readlines()

# party to the agreement is used here
field_abv = "grouping_abv"
field_grp = "grouping"
field_party = "party"

df_out = []
for ln in text:
    spl = ln.split(":")
    
    nms = spl[0].replace(")", "").split("(")
    abv = nms[0].strip()
    nm = nms[1].strip()
    
    parties = spl[1].replace("\n", "").split(",")
    parties = [x.strip() for x in parties]
    
    df = pd.DataFrame(
        {
            field_party: parties
        }
    )
    df[field_grp] = nm
    df[field_abv] = abv
    
    df_out.append(df)
    
    
df_out = pd.concat(df_out)


# try to merge
regions = sc.Regions(sa.model_attributes)
field_regions_try = f"{regions.key}_try"
df_out[field_regions_try] = df_out[field_party].apply(regions.clean_region)

# manual replacement
dict_repl = {
    "laos_(people’s_democratic_republic)": "lao",
    "cape_verde": "cabo_verde",
    #"saint_pierre_and_miquelon": 
    "congo_(the_democratic_republic_of_the)": "democratic_republic_of_the_congo",
    "congo": "republic_of_the_congo",
    "czech_republic": "czechia",
    "russian_federation": "russia",
    "cote_d’ivoire": "cote_divoire",
    "isleofman": "isle_of_man",
    "korea_(democratic_people’s_republic_of)": "democratic_peoples_republic_of_korea",
    "macedonia": "north_macedonia",
    "micronesia_(federated_states_of)": "micronesia",
    "moldova_(republic_of)": "republic_of_moldova",
    "south_korea": "republic_of_korea",
    "swaziland": "eswatini",
    "tanzania": "united_republic_of_tanzania",
    "united_states": "united_states_of_america",
    "us_virgin_islands": "united_states_virgin_islands",
    #"sint_maarten": "saint_martin",
}
df_out[field_regions_try].replace(dict_repl, inplace = True)


In [86]:
tab = regions.attributes.table
tab[~tab["region"].isin(list(df_out[field_regions_try]))]["region"]

85        hong_kong
102          kosovo
114           macao
165    saint_martin
Name: region, dtype: object

In [87]:
dict_rt_to_grp = sf.build_dict(
    df_out[[field_regions_try, field_grp]]
)

# manually map some missing regions to assumed IPCC regions
dict_update_grouping_map = {
    "hong_kong": "East Asia",
    "kosovo": "Economies in Transition",
    "macao": "East Asia",
    "saint_martin": "Latin America and Caribbean",
}

dict_rt_to_grp.update(dict_update_grouping_map)

# groups defined in AR5
grps_ar5 = sorted(list(set(dict_rt_to_grp.values())))

In [105]:
grps_ar5

['East Asia',
 'Economies in Transition',
 'Latin America and Caribbean',
 'Middle East and North Africa',
 'North America',
 'Pacific OECD',
 'South Asia',
 'South-East Asia and Pacific',
 'Sub Saharan Africa',
 'Western Europe']

# Create a preliminary mapping of AR5 categories that probably map to IPCCV4 categories

In [117]:
dict_ar5_to_ipccv4 = {
    "East Asia": "East Asia and South-East Asia",
    "Economies in Transition": "Eastern Europe",
    "Latin America and Caribbean": "Latin America and the Caribbean",
    "Middle East and North Africa": "Near East (Middle East) and North Africa",
    "North America": "North America",
    "South Asia": "South Asia (Indian subcontinent)",
    "Sub Saharan Africa": "Sub-Saharan Africa",
    "Western Europe": "Western Europe",
}
df_merge_high_confidence = pd.DataFrame(dict_ar5_to_ipccv4.items(), columns = ["ar5_region", "ipcc_v4_group"])

# merge into existing regions, then manually merge
tab_out = tab[tab[regions.key].isin(dict_rt_to_grp.keys())][[regions.key]]
tab_out["ar5_region"] = tab[regions.key].replace(dict_rt_to_grp)
tab_out = pd.merge(tab_out, df_merge_high_confidence, how = "left")

tab_out.to_csv("/Users/jsyme/Desktop/tmp_region_mapping.csv", index = None, encoding = "UTF-8")



In [106]:
df_ipcc_fracs = pd.read_csv(
    "/Users/jsyme/SISEPUEDE_DATA_REPOSITORY/ipcc_gnghgi_tables/ipcc2019r_v4c10_table_10a.6-10a.9.csv"
)
sorted(list(df_ipcc_fracs["Climate and System Based Category"].unique()))

['East Asia and South-East Asia',
 'Eastern Europe',
 'Eastern Europe (including Russia)',
 'Latin America and the Caribbean',
 'Near East (Middle East) and North Africa',
 'North America',
 'Oceania',
 'Russia',
 'South Asia (Indian subcontinent)',
 'Sub-Saharan Africa',
 'Western Europe']

# Read the manually-shifted in and mege to region attribute table


In [131]:
df_ipcc_regions = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/ipcc_region_mapping.csv"
df_ipcc_regions = pd.read_csv(df_ipcc_regions).rename(
    columns = {
        "region": "``$REGION$``",
        "ar5_region": "IPCC AR5 Region",
        "ipcc_v4_group": "IPCC Estimated AFOLU Region"
    }
)

In [136]:
attr_new = pd.read_csv("/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs/attribute_region.csv")
attr_new.columns

Index(['Category Name', '``$REGION$``', 'ISO Alpha 3', 'ISO Alpha 2',
       'ISO Numeric', 'FAO Area Code', 'UN Region', 'UN Sub-Region',
       'World Bank Global Region', 'Latitude Population Centroid 2020',
       'Longitude Population Centroid 2020'],
      dtype='object')

In [138]:
region_attribute_new.columns

Index(['Category Name', '``$REGION$``', 'ISO Alpha 3', 'ISO Alpha 2',
       'ISO Numeric', 'FAO Area Code', 'UN Region', 'UN Sub-Region',
       'World Bank Global Region', 'Latitude Population Centroid 2020',
       'Longitude Population Centroid 2020', 'IPCC AR5 Region',
       'IPCC Estimated AFOLU Region'],
      dtype='object')

In [145]:
region_attribute_new = pd.merge(
    attr_new,
    df_ipcc_regions,
    how = "inner"
)

fields_ord = [
    "Category Name",
    "``$REGION$``",
    "ISO Alpha 3",
    "ISO Alpha 2",
    "ISO Numeric",
    "FAO Area Code", 
    "IPCC AR5 Region",
    "IPCC Estimated AFOLU Region",
    "UN Region",
    "UN Sub-Region",
    "World Bank Global Region",
    "Latitude Population Centroid 2020",
    "Longitude Population Centroid 2020"
]

region_attribute_new = region_attribute_new[fields_ord]

In [148]:
region_attribute_new.to_csv(
    "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs/attribute_region.csv",
    index = None,
    encoding = "UTF-8"
)